In [1]:
from pymavlink import mavutil
import math
import tkinter as tk
import socket
import time
import helper
import threading
import torch
import torch.nn as nn
import numpy as np
import random
import transformations as tft
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [2]:
SIM_COMPUTER_IP = '192.168.1.110'  # IP address of the simulation computer
PORT = 15000  # The same port as used by the server

In [3]:
def send_command(command):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((SIM_COMPUTER_IP, PORT))
        s.sendall(command.encode('utf-8'))

def start_instance(instance_id, out_port):
    send_command(f"start {instance_id} {out_port}")

def stop_instance(instance_id):
    send_command(f"stop {instance_id}")

def connect(port):
    connection = mavutil.mavlink_connection(f'udpin:0.0.0.0:{port}') 

    connection.wait_heartbeat() #wait until we hear a heartbeat from the copter

    return connection 

def get_current_position(connection):

    msg = connection.recv_match(type='LOCAL_POSITION_NED', blocking=True)

    north = msg.x
    east = msg.y
    down = -msg.z

    return(north, east, down)


def set_attitude(connection, x, y, z, r):
    q = tft.quaternion_from_euler(x, y, r)
    
    # Send SET_ATTITUDE_TARGET message

    connection.mav.set_attitude_target_send(
        0,  # time_boot_ms
        connection.target_system,
        connection.target_component,
        0b00000000,  # Type mask
        q,  # Quaternion
        0, 0, 0,  # Body roll, pitch, yaw rate
        z # Thrust

    )
    
def get_current_attitude(connection):
    attitude_msg = connection.recv_match(type='ATTITUDE', blocking=True)
    hud_msg = connection.recv_match(type='VFR_HUD', blocking=True)
    if attitude_msg is not None:
        roll = attitude_msg.roll
        pitch = attitude_msg.pitch
        yaw_rate = attitude_msg.yawspeed 

    if hud_msg is not None:
        thrust = hud_msg.throttle 

    print(f"Roll: {roll:.3f} rad, Pitch: {pitch:.3f} rad, Yaw Rate: {yaw_rate:.3f} rad/s, Thrust: {thrust}%")
    return (roll, pitch, yaw_rate, thrust)

def wait_for_gps_lock(connection, timeout=60):
    start_time = time.time()
    while True:
        # Check for timeout
        if time.time() - start_time > timeout:
            print("Timeout waiting for GPS lock.")
            return False
            
        # Fetch GPS_RAW_INT messages
        msg = connection.recv_match(type='GPS_RAW_INT', blocking=True, timeout=5)
        if not msg:
            print("No GPS data received.")
            continue
            
        # Check for 3D lock
        if msg.fix_type >= 3:
            print(f"GPS lock acquired with {msg.satellites_visible} satellites.")
            return True
        else:
            print(f"Current GPS fix type: {msg.fix_type}, waiting for 3D lock...")

        time.sleep(1)


def arm(mavlink_connection):
    """
    Arms vehicle and fly to a target altitude.
    :param mavlink_connection: The connection to the vehicle
    :param target_altitude: Target altitude in meters
    """

    print("Basic pre-arm checks")
    # Wait for vehicle to initialize and become ready
    while not mavlink_connection.wait_heartbeat(timeout=5):
        print("Waiting for vehicle heartbeat")

           
    print("Setting vehicle to GUIDED mode")
    mavlink_connection.mav.set_mode_send(mavlink_connection.target_system, mavutil.mavlink.MAV_MODE_FLAG_CUSTOM_MODE_ENABLED, 4) 
    
    

    # Wait a bit for the mode to change
    time.sleep(2)

    # Copter should arm in GUIDED mode
    mavlink_connection.mav.command_long_send(
        mavlink_connection.target_system, mavlink_connection.target_component,
        mavutil.mavlink.MAV_CMD_COMPONENT_ARM_DISARM,
        0,  # Confirmation
        1,  # 1 to arm
        0, 0, 0, 0, 0, 0  # Unused parameters for this command
    )
    
    
def waitForEKF2(connection):
    while True:
        msg = connection.recv_match(type='STATUSTEXT', blocking=True, timeout=5)
        if msg is None:
            continue
        text = msg.text.decode('utf-8')
        if "EKF2 IMU0 is using GPS" in text:
            print("Received EKF2 GPS lock message.")
            return True

def takeoff(mavlink_connection, alt):
    #Assumes you have already set to guided mode and armed.

    mavlink_connection.mav.command_long_send(mavlink_connection.target_system, mavlink_connection.target_component, 
                                 mavutil.mavlink.MAV_CMD_NAV_TAKEOFF, 0, 0, 0, 0, 0, 0, 0, alt)
    
    #See how the copter responded to the takeoff command
    msg = mavlink_connection.recv_match(type = 'COMMAND_ACK', blocking = True)
    #print(msg)

    while True:
        # Wait for the next LOCAL_POSITION_NED message
        msg = mavlink_connection.recv_match(type='LOCAL_POSITION_NED', blocking=True)
        
    
        # Check if altitude is within a threshold of the target altitude
        if abs(msg.z * -1 - alt) < 1.0:
            print("Reached target altitude")
            break

#send local frame coordinates and have copter fly over that spot.
def send_waypoint_local(connection, x, y, alt):
    connection.mav.send(mavutil.mavlink.MAVLink_set_position_target_local_ned_message
                    (10, connection.target_system, connection.target_component, mavutil.mavlink.MAV_FRAME_LOCAL_NED, 
                     int(0b010111111000), x, y, alt,
                      0, 0, 0, 0, 0, 0, 0, 0))

    time.sleep(1)


In [4]:
def plot_3d_paths(results):
    # Create a new figure for plotting
    fig = plt.figure(figsize=(15, 13))
    
    # Determine the number of subplots based on the number of paths
    num_paths = len(results)
    cols = 2  # You can adjust the number of columns based on your preference
    rows = (num_paths + cols - 1) // cols  # Calculate rows needed
    
    x_limits = (-10, 10)
    y_limits = (-10, 10)
    z_limits = (0, 10)
    
    # Plot each path in a subplot
    for index, (pid, path) in enumerate(results.items(), start=1):
        ax = fig.add_subplot(rows, cols, index, projection='3d')
        # Assuming path is a list of [north, east, down] coordinates
        path = np.array(path)  # Convert path to a NumPy array for easier slicing
        
        # Create a color map
        points = np.arange(len(path))
        colors = cm.plasma(points / max(points))  # Normalize points for color mapping
        
        # Plot the 3D path with a color gradient
        ax.scatter(path[:, 0], path[:, 1], path[:, 2], c=colors, cmap='plasma', marker='o')
        
        # Plot the target point in red
        ax.plot([7], [7], [7], marker='o', color='red', markersize=10)
        
        ax.set_xlim(x_limits)
        ax.set_ylim(y_limits)
        ax.set_zlim(z_limits)
        
        ax.set_xlabel('North')
        ax.set_ylabel('East')
        ax.set_zlabel('Down')
        ax.set_title(f'PID {pid} Flight Path')

    plt.tight_layout()
    plt.show()


In [5]:
# GANN definitions #

class DroneNet(nn.Module):
    def __init__(self):
        super(DroneNet, self).__init__()
        self.fc1 = nn.Linear(7, 10)  # First hidden layer from 7 inputs to 10 nodes
        self.fc2 = nn.Linear(10, 5)  # Second hidden layer from 10 nodes to 5 nodes
        self.output = nn.Linear(5, 4)  # Output layer from 5 nodes to 4 outputs (roll, pitch, thrust, yaw_rate)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLu activation for first hidden layer
        x = torch.relu(self.fc2(x))  # ReLu activation for second hidden layer
        x = self.output(x)  # Output layer, no activation (linear output)
        return x
    
def get_fitness(path, target):
    # calculate how close final position was to target and the total distance.
    # return a fitness that rewards closeness to target and minumum distance flown.
    final_position = path[-1]
    fp_north, fp_east, fp_down = final_position
    t_north, t_east, t_down = target

    distance_from_target = np.sqrt((fp_north-t_north)**2 + (fp_east-t_east)**2 + (fp_down-t_down)**2)

    total_distance = 0
    for i in range(1, len(path)):
        n, e, d = path[i]
        old_n, old_e, old_d = path[i-1]

        total_distance += np.sqrt((old_n-n)**2 + (old_e-e)**2 + (old_d-d)**2)
        
    distance_target_score = 1 / (distance_from_target + 1)
    distance_total_score = -1 / (total_distance + 1) 
    
    if distance_from_target > 20:
        distance_target_score = distance_target_score - 0.75
        
    distance_target_score = 1 / (distance_from_target + 1)
    
    fitness = distance_target_score
    
    print(f"Fitness: {fitness} dist_from_target = {distance_from_target} and total_distace = {total_distance}")
    
    return fitness

def mutate(network, mutation_rate, mutation_strength):
    with torch.no_grad():
        for param in network.parameters():
            if random.random() < mutation_rate: 
                mutation = torch.randn_like(param) * mutation_strength #create a tensor in the same shape as the parameter with random integers scaled by the strenght
                param.add_(mutation) #add to existing parameter

def crossover(parent1, parent2):
    child = DroneNet() 
    with torch.no_grad():
        for child_param, param1, param2 in zip(child.parameters(), parent1.parameters(), parent2.parameters()):
            mask = torch.bernoulli(torch.full_like(param1, 0.5))  # Randomly take weights from either parent
            child_param.data.copy_(param1.data * mask + param2.data * (1 - mask))
    return child

def initialize_population(size):
    return [DroneNet() for _ in range(size)]

def generate_new_population(population, fitness_scores, mutation_rate, mutation_strength):
    new_population = []
    population_size = len(population)
    
    sorted_indices = sorted(range(len(fitness_scores)), key=lambda k: fitness_scores[k], reverse=True)
        
    # Two highest fitnesses
    elite = [population[i] for i in sorted_indices[:2]]

    # Crossover and mutation to fill the new population
    while len(new_population) < population_size - len(elite):
        parent1, parent2 = random.sample(elite, 2) # leaving like this in case I increase number of elites 
        child = crossover(parent1, parent2)
        mutate(child, mutation_rate, mutation_strength)
        new_population.append(child)

    # Ensure that the total new population size does not exceed the original population size
    new_population.extend(elite[:population_size - len(new_population)])

    return new_population

def adjust_output(output):
    # Scale from [0, 1] to [1, 50]
    return 1 + 49 * output


In [6]:
def simulate_drone(pid, out_port, network, result_dict, results_lock):
    try:
        print(f"starting instance {pid} on port {out_port}")
        start_instance(pid, out_port)
        drone_connection = connect(out_port)
        
        while True:
            msg = drone_connection.recv_match(type='HEARTBEAT', blocking=True)
            if not msg:
                print(f"instance {pid} - No heartbeat")
            else:
                print(f"instance {pid} - Heartbeat Recieved!")
                break
            
        

        time.sleep(7)

        print(f"instance {pid} - arming throttle in GUIDED mode.")
        arm(drone_connection)

        print(f"instance {pid} - sending takeoff command")
        takeoff(drone_connection, 10)

        
        #time.sleep(4)

        path = []

        for i in range(15):
            north, east, down = get_current_position(drone_connection)
            roll, pitch, thrust, yaw_rate = get_current_attitude(drone_connection)

            #print(f'Instance {pid} - North: {north}, East: {east}, Down: {down}')
            
            current_state = [north, east, down, roll, pitch, yaw_rate, thrust]
            current_pos = [north, east, down]
            
            path.append(current_pos)

            with torch.no_grad():
                input_tensor = torch.tensor(current_state, dtype=torch.float32).unsqueeze(0)
                new_attitude = network(input_tensor).numpy()[0]  # Remove batch dimension and convert to numpy
                print(f"Debug - New Attitude: {new_attitude}")
                new_roll, new_pitch, new_thrust, new_yaw_rate = new_attitude
                

            #print(f"Instance {pid} sending neurotic waypoint: {new_north}, {new_east}, {new_down}")

            set_attitude(drone_connection, new_roll, new_pitch, new_thrust, new_yaw_rate)
            time.sleep(0.01)
        

        print(f"stopping instance {pid}")
        stop_instance(pid)

        result_dict[pid] = path
    
    except Exception as e:
        print(f"Exception in thread {pid}: {e}")


In [7]:
def main():
    out_port_base = 11000
    pid_base = 0
    instance_count = 6  # Number of individuals (simulation instances) per cycle
    epochs = 50
    target = [7, 7, 7] 
    mutation_rate = 0.3
    mutation_strength = 0.1
    num_cycles = 3
    population_size = instance_count * num_cycles
       
    print("Testing fixed fitness_scores indexing")
    print(f"Mutation rate is {mutation_rate} and mutation strength is {mutation_strength}")

    # Initialize the population
    population = initialize_population(population_size)
    
    for j in range(epochs):
        print(f"<----------------------- EPOCH {j} ------------------------->")
        fitness_scores = [0] * population_size
        
        for cycle in range (num_cycles):
            print(f"|- - - - - - - - - - - - cycle {cycle} - - - - - - - - - - - - -|")
            
            threads = []
            results = {}
            results_lock = threading.Lock()
            
            start_index = cycle * instance_count
            end_index = min((cycle + 1) * instance_count, population_size)

            for i in range(start_index, end_index):
                pid = pid_base + i
                out_port = out_port_base + i
                network = population[i]
                # Passing results_lock to ensure it is used in simulate_drone
                t = threading.Thread(target=simulate_drone, args=(pid, out_port, network, results, results_lock))
                t.start()
                threads.append(t)

            for t in threads:
                t.join(timeout=30)

            with results_lock:
                for pid, path in results.items():
                    index = pid % population_size
                    fitness = get_fitness(path, target)
                    fitness_scores[index] = fitness
                    print(f"Instance: {pid} fitness: {fitness}")

            plot_3d_paths(results)
                
            pid_base += instance_count
            out_port_base += instance_count   
            if (out_port_base >= 11120):
                out_port_base = 11000

        population = generate_new_population(population, fitness_scores, mutation_rate, mutation_strength)

       
            
        
if __name__ == "__main__":
    main() 

Testing fixed fitness_scores indexing
Mutation rate is 0.3 and mutation strength is 0.1
<----------------------- EPOCH 0 ------------------------->
|- - - - - - - - - - - - cycle 0 - - - - - - - - - - - - -|
starting instance 0 on port 11000
starting instance 1 on port 11001
starting instance 2 on port 11002
starting instance 3 on port 11003
starting instance 4 on port 11004
starting instance 5 on port 11005
instance 2 - Heartbeat Recieved!
instance 0 - Heartbeat Recieved!
instance 3 - Heartbeat Recieved!
instance 1 - Heartbeat Recieved!
instance 5 - Heartbeat Recieved!
instance 4 - Heartbeat Recieved!
instance 2 - arming throttle in GUIDED mode.
Basic pre-arm checks
Setting vehicle to GUIDED mode
instance 0 - arming throttle in GUIDED mode.
Basic pre-arm checks
Setting vehicle to GUIDED mode
instance 3 - arming throttle in GUIDED mode.
Basic pre-arm checks
Setting vehicle to GUIDED mode
instance 1 - arming throttle in GUIDED mode.
Basic pre-arm checks
Setting vehicle to GUIDED mode
in

<Figure size 1500x1300 with 0 Axes>

|- - - - - - - - - - - - cycle 1 - - - - - - - - - - - - -|
starting instance 12 on port 11012
starting instance 13 on port 11013
starting instance 14 on port 11014
starting instance 15 on port 11015
starting instance 16 on port 11016
starting instance 17 on port 11017
instance 17 - Heartbeat Recieved!
instance 14 - Heartbeat Recieved!
instance 15 - Heartbeat Recieved!
instance 16 - Heartbeat Recieved!
instance 17 - arming throttle in GUIDED mode.
Basic pre-arm checks
Setting vehicle to GUIDED mode
instance 14 - arming throttle in GUIDED mode.
Basic pre-arm checks
Setting vehicle to GUIDED mode
instance 15 - arming throttle in GUIDED mode.
Basic pre-arm checks
Setting vehicle to GUIDED mode
instance 16 - arming throttle in GUIDED mode.
Basic pre-arm checks
Setting vehicle to GUIDED mode
instance 17 - sending takeoff command
instance 14 - sending takeoff command
instance 15 - sending takeoff command
instance 16 - sending takeoff command
